# Auto.ria data cleaning


In [1]:
import os
import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
cwd = Path(os.getcwd())
data_path = cwd.parent/'datasets/autoria/autoria_data.csv'
data_path.exists()

True

In [3]:
df = pd.read_csv(data_path, sep='|', encoding='cp1251')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146814 entries, 0 to 146813
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 146814 non-null  int64 
 1   brand              146814 non-null  object
 2   price              146814 non-null  int64 
 3   currency           146814 non-null  object
 4   mileage            146814 non-null  object
 5   fuel_type          146814 non-null  object
 6   transmission_type  146814 non-null  object
 7   pub_date           146814 non-null  object
dtypes: int64(2), object(6)
memory usage: 9.0+ MB


In [4]:
df.head()

,id,brand,price,currency,mileage,fuel_type,transmission_type,pub_date
0,0,Mercedes-Benz E 280 4MATIC AT CLASSIC 2007,8999,USD,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021
1,1,Mercedes-Benz G 350 Designo 2015,79900,USD,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021
2,2,BMW 550 2010,15999,USD,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021
3,3,MINI Countryman S 2014,10200,USD,111 тис. км,"Бензин, 1.6 л.",Автомат,сьогодні о 12:00
4,4,Nissan X-Trail 2010,13900,USD,97 тис. км,"Бензин, 2 л.",Автомат,19.05.2021


### brand column

It seems like `brand` column can be split into 3:<br> 
`brand` - first part,<br> 
`car model` - everithing between first and last parts and<br> 
`year made` - the last part.

In [5]:
df['brand'].str.split()

0         [Mercedes-Benz, E, 280, 4MATIC, AT, CLASSIC, 2...
1                    [Mercedes-Benz, G, 350, Designo, 2015]
2                                          [BMW, 550, 2010]
3                               [MINI, Countryman, S, 2014]
4                                   [Nissan, X-Trail, 2010]
                                ...                        
146809                       [Renault, Kangoo, пасс., 2008]
146810                             [Audi, 80, 1.8, S, 1988]
146811                                [BMW, 520, F10, 2011]
146812                                    [ВАЗ, 1118, 2006]
146813                                [Nissan, Rogue, 2017]
Name: brand, Length: 146814, dtype: object

In [6]:
year_made = df['brand'].str.split().str[-1]
year_made.head()

0    2007
1    2015
2    2010
3    2014
4    2010
Name: brand, dtype: object

In [7]:
pd.to_numeric(year_made).describe()

count    146814.000000
mean       2008.295994
std           7.541967
min        1900.000000
25%        2005.000000
50%        2009.000000
75%        2013.000000
max        2021.000000
Name: brand, dtype: float64

In [8]:
df["year_made"] = pd.to_numeric(year_made)
df.head(3)

,id,brand,price,currency,mileage,fuel_type,transmission_type,pub_date,year_made
0,0,Mercedes-Benz E 280 4MATIC AT CLASSIC 2007,8999,USD,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007
1,1,Mercedes-Benz G 350 Designo 2015,79900,USD,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021,2015
2,2,BMW 550 2010,15999,USD,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021,2010


In [9]:
# Remove year from the brand column
df["brand"] = df.brand.str.rsplit(n=1).str.get(0)
df.head(3)

,id,brand,price,currency,mileage,fuel_type,transmission_type,pub_date,year_made
0,0,Mercedes-Benz E 280 4MATIC AT CLASSIC,8999,USD,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007
1,1,Mercedes-Benz G 350 Designo,79900,USD,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021,2015
2,2,BMW 550,15999,USD,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021,2010


In [10]:
# Split car brand and model into seperate columns
df["model"] = df.brand.str.split(n=1).str.get(1)
df["brand"] = df.brand.str.split(n=1).str.get(0)
df.head(3)

,id,brand,price,currency,mileage,fuel_type,transmission_type,pub_date,year_made,model
0,0,Mercedes-Benz,8999,USD,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC
1,1,Mercedes-Benz,79900,USD,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021,2015,G 350 Designo
2,2,BMW,15999,USD,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021,2010,550


### price and currency columns

It would be best for future modeling to have single currency for price.<br>
So let's convert all prices to USD and remove currency column.

In [11]:
df['currency'].value_counts()

USD    145519
UAH       688
EUR       607
Name: currency, dtype: int64

In [12]:
# Prices before conversion
df['price'].describe()

count    1.468140e+05
mean     1.269740e+04
std      3.546551e+04
min      2.500000e+02
25%      5.150000e+03
50%      8.000000e+03
75%      1.320000e+04
max      3.172743e+06
Name: price, dtype: float64

In [13]:
# Convert currencies other than USD to USD
UAH_USD = 0.036
EUR_USD = 1.22

df.loc[df['currency'] == 'UAH', 'price'] *= UAH_USD
df.loc[df['currency'] == 'EUR', 'price'] *= EUR_USD

df['price'] = df['price'].astype(int)

# Prices after conversion
df['price'].describe()

count    146814.000000
mean      11559.182421
std       16175.555228
min         250.000000
25%        5100.000000
50%        8000.000000
75%       13000.000000
max      529900.000000
Name: price, dtype: float64

In [14]:
# Rename price column and remove currency
df.rename(columns={'price': 'price_USD'}, inplace=True)
df.drop(columns=['currency'], inplace=True)
df.head(3)

,id,brand,price_USD,mileage,fuel_type,transmission_type,pub_date,year_made,model
0,0,Mercedes-Benz,8999,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC
1,1,Mercedes-Benz,79900,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021,2015,G 350 Designo
2,2,BMW,15999,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021,2010,550


### mileage column

In [15]:
df.head(3)

,id,brand,price_USD,mileage,fuel_type,transmission_type,pub_date,year_made,model
0,0,Mercedes-Benz,8999,159 тис. км,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC
1,1,Mercedes-Benz,79900,103 тис. км,"Дизель, 3 л.",Автомат,15.05.2021,2015,G 350 Designo
2,2,BMW,15999,145 тис. км,"Бензин, 4.4 л.",Автомат,18.04.2021,2010,550


In [16]:
# Check formats diversity for 'mileage'
df['mileage'].str.split().str.len().value_counts()

3    143836
2      2978
Name: mileage, dtype: int64

In [17]:
# 'без пробега (no mileage)' must mean that car is fresh 
# and haven't been driven, so it will be reasonable to replace it with zeros.

df.loc[df['mileage'].str.split().str.len() == 2, 'mileage'].value_counts()

 без пробега     2978
Name: mileage, dtype: int64

In [18]:
# All non zero mileage values are mesured in 'тис. км (thousand kilometers)'
# So we can just drop that part and leave only numeric one.

df.mileage.str.split(n=1).str[1].value_counts()

тис. км     143836
пробега       2978
Name: mileage, dtype: int64

In [19]:
# drop 'тис. км' part
df.loc[df.mileage.str.split().str.len() == 3, 'mileage'] = df.mileage.str.split().str.get(0)
# set to zero all 'без пробега' values
df.loc[df.mileage.str.split().str.len() == 2, 'mileage'] = 0
df['mileage'].head(3)

0    159
1    103
2    145
Name: mileage, dtype: object

In [20]:
# Convert to int
df["mileage"] = pd.to_numeric(df["mileage"])
df["mileage"].head()

0    159
1    103
2    145
3    111
4     97
Name: mileage, dtype: int64

In [21]:
# Rename 'mileage' to 'mileage_kkm' not to forget that values in this column are 1000km
df.rename(columns={'mileage': 'mileage_kkm'}, inplace=True)
df.head(3)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model
0,0,Mercedes-Benz,8999,159,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC
1,1,Mercedes-Benz,79900,103,"Дизель, 3 л.",Автомат,15.05.2021,2015,G 350 Designo
2,2,BMW,15999,145,"Бензин, 4.4 л.",Автомат,18.04.2021,2010,550


In [22]:
df['mileage_kkm'].describe()

count    146814.000000
mean        186.485158
std         982.145206
min           0.000000
25%         113.000000
50%         180.000000
75%         236.000000
max      239000.000000
Name: mileage_kkm, dtype: float64

In [23]:
df[df['mileage_kkm'] > 1000]

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model
64257,64257,Lexus,21600,195000,"Газ / Бензин, 4.7 л.",Автомат,11.05.2021,2005,LX 470
93409,93409,BMW,6000,33000,"Газ / Бензин, 3 л.",Автомат,4.05.2021,1996,730
144533,144533,ВАЗ,1500,239000,"Газ / Бензин, 1.9 л.",Ручна / Механіка,16.05.2021,1989,2106 21061 1.5
144968,144968,Fiat,4500,210000,"Бензин, 1.2 л.",Ручна / Механіка,15.05.2021,2003,Doblo груз.


According to [Wikipedia](https://en.wikipedia.org/wiki/Car_longevity):

`typical car lasts closer to 322,000 kilometres`<br>
So it should be safe to assume that mileage over 1,000,000 kilometers is incorrect<br>
(most likely users entered mileage data in plain kilometers)

In [24]:
# In order to normalize outliers let's divide this values by 1000
df.loc[df['mileage_kkm'] > 1000, 'mileage_kkm'] //= 1000
df['mileage_kkm'].describe()

count    146814.000000
mean        181.878493
std         102.133535
min           0.000000
25%         113.000000
50%         180.000000
75%         236.000000
max         999.000000
Name: mileage_kkm, dtype: float64

### fuel_type column

In [25]:
# 'fuel_type' column can be split into 'fuel_type' and 'engine_size' (engine displacement)
df.head(3)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model
0,0,Mercedes-Benz,8999,159,"Газ / Бензин, 3 л.",Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC
1,1,Mercedes-Benz,79900,103,"Дизель, 3 л.",Автомат,15.05.2021,2015,G 350 Designo
2,2,BMW,15999,145,"Бензин, 4.4 л.",Автомат,18.04.2021,2010,550


In [26]:
df['fuel_type'][2]

' Бензин, 4.4 л. '

In [27]:
# Remove spaces
df['fuel_type'] = df['fuel_type'].str.strip()
df['fuel_type'][2]

'Бензин, 4.4 л.'

In [28]:
df['fuel_type'].str.split(',').str.len().value_counts()

2    135182
1     11632
Name: fuel_type, dtype: int64

In [29]:
df.loc[df['fuel_type'].str.split(',').str.len() == 1, 'fuel_type']

17              Дизель
39              Бензин
41              Бензин
49              Бензин
58              Бензин
              ...     
146801    Газ / Бензин
146803          1.9 л.
146804          Дизель
146810          Бензин
146812          Бензин
Name: fuel_type, Length: 11632, dtype: object

So it seems like fuel_type column contains 3 kind of values:<br>
only fuel type, only engine size and both(fuel type and engine size)<br>
Let's seperate them

In [30]:
# Engine size from rows that have both fuel type and engine size
fuel_and_size = df['fuel_type'].str.split(',').str[1]
fuel_and_size.tail(4)

146810        NaN
146811       2 л.
146812        NaN
146813     2.5 л.
Name: fuel_type, dtype: object

In [31]:
# Engine size from columns that have only engine size
size_only = df[df['fuel_type'].str.split(',').str[0].str.strip().str.split().str[1] == 'л.'].fuel_type
size_only.head(3)

552     3.5 л.
945       2 л.
1733    3.5 л.
Name: fuel_type, dtype: object

In [32]:
fuel_and_size.isna().sum()

11632

In [33]:
size_only.size

1001

In [34]:
# Fill NaN's from fuel_and_size with values in size_only
engine_size = fuel_and_size.combine_first(size_only)
engine_size.isna().sum()

10631

In [35]:
# All engine size values are specified in liters
# hence we can leave only numerical part
engine_size.str.split().str[1].value_counts()

л.    136183
Name: fuel_type, dtype: int64

In [36]:
engine_size = pd.to_numeric(engine_size.str.split().str[0])
engine_size.head(3)

0    3.0
1    3.0
2    4.4
Name: fuel_type, dtype: float64

In [37]:
df['engine_size'] = engine_size
df.tail(5)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
146809,146809,Renault,6100,260,"Дизель, 1.5 л.",Ручна / Механіка,10.05.2021,2008,Kangoo пасс.,1.5
146810,146810,Audi,2100,352,Бензин,Ручна / Механіка,10.05.2021,1988,80 1.8 S,NaN
146811,146811,BMW,18000,70,"Бензин, 2 л.",Автомат,10.05.2021,2011,520 F10,2.0
146812,146812,ВАЗ,400,278,Бензин,Ручна / Механіка,10.05.2021,2006,1118,NaN
146813,146813,Nissan,14900,73,"Бензин, 2.5 л.",Автомат,10.05.2021,2017,Rogue,2.5


In [38]:
df['engine_size'].describe()

count    136183.000000
mean          2.022561
std           0.854566
min           0.000000
25%           1.600000
50%           1.900000
75%           2.200000
max          99.990000
Name: engine_size, dtype: float64

In [39]:
# This is definitely wrong
df.loc[df['engine_size'] > 10]

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
16486,16486,ВАЗ,1080,10,"Бензин, 13 л.",Ручна / Механіка,21.05.2021,1986,2101 21013 1.2,13.00
16847,16847,Toyota,73000,2,"Дизель, 11 л.",Автомат,21.05.2021,2008,Land Cruiser 200,11.00
25240,25240,ВАЗ,1450,25,"Бензин, 13 л.",Ручна / Механіка,19.05.2021,1990,2106 21063,13.00
30277,30277,Kia,5600,179,"Газ / Бензин, 16 л.",Ручна / Механіка,18.05.2021,2007,Cerato,16.00
40842,40842,Nissan,3150,250,"Газ / Бензин, 12 л.",Ручна / Механіка,16.05.2021,1995,Maxima,12.00
49916,49916,Daewoo,2500,250,"Газ / Бензин, 13 л.",Ручна / Механіка,14.05.2021,2005,Sens,13.00
58371,58371,Opel,5500,370,"Дизель, 25 л.",Ручна / Механіка,12.05.2021,2006,Movano груз.,25.00
59625,59625,Chevrolet,8350,190,"Дизель, 20 л.",Ручна / Механіка,12.05.2021,2008,Captiva 2.0 CDI 7 MICT,20.00
80468,80468,Fiat,1150,100,"Бензин, 16 л.",Не вказано,7.05.2021,1988,Tipo,16.00
81193,81193,Mazda,6500,120,"Бензин, 20 л.",Ручна / Механіка,7.05.2021,2006,3,20.00


In [40]:
# Set engine_size values bigger than 10 to NaN
# if needed it can be derived from the date
df.loc[df['engine_size'] > 10, 'engine_size'] = np.nan
df['engine_size'].describe()

count    136162.000000
mean          2.018547
std           0.712282
min           0.000000
25%           1.600000
50%           1.900000
75%           2.200000
max           7.400000
Name: engine_size, dtype: float64

In [41]:
# Remove engine size from fuel_type where are both fuel type and engine size
df['fuel_type'] = df['fuel_type'].str.split(',').str[0].str.strip()
df.tail(5)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
146809,146809,Renault,6100,260,Дизель,Ручна / Механіка,10.05.2021,2008,Kangoo пасс.,1.5
146810,146810,Audi,2100,352,Бензин,Ручна / Механіка,10.05.2021,1988,80 1.8 S,NaN
146811,146811,BMW,18000,70,Бензин,Автомат,10.05.2021,2011,520 F10,2.0
146812,146812,ВАЗ,400,278,Бензин,Ручна / Механіка,10.05.2021,2006,1118,NaN
146813,146813,Nissan,14900,73,Бензин,Автомат,10.05.2021,2017,Rogue,2.5


In [42]:
df['fuel_type'].value_counts()

Бензин          62065
Дизель          47038
Газ / Бензин    31568
Електро          1810
Гібрид           1686
                ...  
1.34 л.             1
1.78 л.             1
6.2 л.              1
1.45 л.             1
1.61 л.             1
Name: fuel_type, Length: 72, dtype: int64

In [43]:
# Set fuel_type column values to NaN where is no fuel type only engine size
df.loc[df['fuel_type'].str.contains('л.', regex=False), 'fuel_type'] = np.nan
df['fuel_type'].value_counts()

Бензин              62065
Дизель              47038
Газ / Бензин        31568
Електро              1810
Гібрид               1686
Не вказано            834
Газ                   535
Гибрид                124
Газ пропан-бутан       62
Электро                42
Газ метан              42
Інше                    7
Name: fuel_type, dtype: int64

In [44]:
# Normalize category names in fuel_type column

ftype_replace_dict = {
    'fuel_type': {
        'Бензин': 'petrol',
        'Дизель': 'diesel',
        'Газ / Бензин': 'gas/petrol',
        
        'Електро': 'electric',
        'Электро': 'electric',
        
        'Гібрид': 'hybrid',
        'Гибрид': 'hybrid',
        
        'Не вказано': np.nan,
        'Інше': np.nan,
        
        'Газ': 'gas',
        'Газ пропан-бутан': 'gas',
        'Газ метан': 'gas'
    }
}


df.replace(ftype_replace_dict, inplace=True)
df['fuel_type'].value_counts()

petrol        62065
diesel        47038
gas/petrol    31568
electric       1852
hybrid         1810
gas             639
Name: fuel_type, dtype: int64

### transmission_type column

In [45]:
df.head(3)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
0,0,Mercedes-Benz,8999,159,gas/petrol,Автомат,21.05.2021,2007,E 280 4MATIC AT CLASSIC,3.0
1,1,Mercedes-Benz,79900,103,diesel,Автомат,15.05.2021,2015,G 350 Designo,3.0
2,2,BMW,15999,145,petrol,Автомат,18.04.2021,2010,550,4.4


In [46]:
df['transmission_type'][0]

' Автомат '

In [47]:
# Remove spaces
df['transmission_type'] = df['transmission_type'].str.strip()

In [48]:
df['transmission_type'][0]

'Автомат'

In [49]:
df['transmission_type'].value_counts()

Ручна / Механіка    74410
Автомат             53876
Не вказано           8930
Типтронік            3725
Варіатор             2770
Робот                1901
Механическая          705
Вариатор              418
Роботизированная       53
Типтроник              26
Name: transmission_type, dtype: int64

In [50]:
# Normalize category names in transmission_type column

ttype_replace_dict = {
    'transmission_type': {
        'Ручна / Механіка': 'manual',
        'Механическая': 'manual',
        
        'Автомат': 'automatic',
        'Не вказано': np.nan,
        
        'Типтронік': 'tiptronic',
        'Типтроник': 'tiptronic',
        
        'Варіатор': 'variator',
        'Вариатор': 'variator',
        
        'Робот': 'robotic',
        'Роботизированная': 'robotic',
        
    }
}

df.replace(ttype_replace_dict, inplace=True)
df['transmission_type'].value_counts()

manual       75115
automatic    53876
tiptronic     3751
variator      3188
robotic       1954
Name: transmission_type, dtype: int64

### pub_date column

In [51]:
df.head()

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
0,0,Mercedes-Benz,8999,159,gas/petrol,automatic,21.05.2021,2007,E 280 4MATIC AT CLASSIC,3.0
1,1,Mercedes-Benz,79900,103,diesel,automatic,15.05.2021,2015,G 350 Designo,3.0
2,2,BMW,15999,145,petrol,automatic,18.04.2021,2010,550,4.4
3,3,MINI,10200,111,petrol,automatic,сьогодні о 12:00,2014,Countryman S,1.6
4,4,Nissan,13900,97,petrol,automatic,19.05.2021,2010,X-Trail,2.0


In [52]:
df['pub_date'][0]

'   21.05.2021          '

In [53]:
# Remove spaces
df['pub_date'] = df['pub_date'].str.strip()

In [54]:
df['pub_date'][0]

'21.05.2021'

In [55]:
# Find inconsistencies in 'pub_date'
df['pub_date'].str.split('.').str.len().value_counts()

3    146811
1         3
Name: pub_date, dtype: int64

In [56]:
df[df['pub_date'].str.split('.').str.len() == 1]

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
3,3,MINI,10200,111,petrol,automatic,сьогодні о 12:00,2014,Countryman S,1.6
23290,23290,Subaru,900,175,petrol,manual,Продано,1988,Leone 4WD Turbo,1.8
23538,23538,Skoda,6600,186,petrol,manual,Продано,2005,Octavia A5 MPI,1.6


In [57]:
# Normalize 'pub_date' values
df.replace({'pub_date': {
    'сьогодні о 12:00': '21.05.2021',
    'Продано': np.nan
}}, inplace=True)

df['pub_date'].str.split('.').str.len().value_counts()

3.0    146812
Name: pub_date, dtype: int64

In [58]:
df['pub_date'] = pd.to_datetime(df['pub_date'])
df.head(3)

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
0,0,Mercedes-Benz,8999,159,gas/petrol,automatic,2021-05-21,2007,E 280 4MATIC AT CLASSIC,3.0
1,1,Mercedes-Benz,79900,103,diesel,automatic,2021-05-15,2015,G 350 Designo,3.0
2,2,BMW,15999,145,petrol,automatic,2021-04-18,2010,550,4.4


### Examine and save cleaned data

In [59]:
df.describe()

,id,price_USD,mileage_kkm,year_made,engine_size
count,146814.000000,146814.000000,146814.000000,146814.000000,136162.000000
mean,73406.500000,11559.182421,181.878493,2008.295994,2.018547
std,42381.695548,16175.555228,102.133535,7.541967,0.712282
min,0.000000,250.000000,0.000000,1900.000000,0.000000
25%,36703.250000,5100.000000,113.000000,2005.000000,1.600000
50%,73406.500000,8000.000000,180.000000,2009.000000,1.900000
75%,110109.750000,13000.000000,236.000000,2013.000000,2.200000
max,146813.000000,529900.000000,999.000000,2021.000000,7.400000


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146814 entries, 0 to 146813
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 146814 non-null  int64         
 1   brand              146814 non-null  object        
 2   price_USD          146814 non-null  int32         
 3   mileage_kkm        146814 non-null  int64         
 4   fuel_type          144972 non-null  object        
 5   transmission_type  137884 non-null  object        
 6   pub_date           146812 non-null  datetime64[ns]
 7   year_made          146814 non-null  int64         
 8   model              146814 non-null  object        
 9   engine_size        136162 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3), object(4)
memory usage: 10.6+ MB


In [61]:
clean_data_path = data_path.parent/'autoria_clean_data.csv'
df.to_csv(clean_data_path, index=False)

In [62]:
# Test saves file reads properly
clean_df = pd.read_csv(clean_data_path)
clean_df

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
0,0,Mercedes-Benz,8999,159,gas/petrol,automatic,2021-05-21,2007,E 280 4MATIC AT CLASSIC,3.0
1,1,Mercedes-Benz,79900,103,diesel,automatic,2021-05-15,2015,G 350 Designo,3.0
2,2,BMW,15999,145,petrol,automatic,2021-04-18,2010,550,4.4
3,3,MINI,10200,111,petrol,automatic,2021-05-21,2014,Countryman S,1.6
4,4,Nissan,13900,97,petrol,automatic,2021-05-19,2010,X-Trail,2.0
...,...,...,...,...,...,...,...,...,...,...
146809,146809,Renault,6100,260,diesel,manual,2021-10-05,2008,Kangoo пасс.,1.5
146810,146810,Audi,2100,352,petrol,manual,2021-10-05,1988,80 1.8 S,NaN
146811,146811,BMW,18000,70,petrol,automatic,2021-10-05,2011,520 F10,2.0
146812,146812,ВАЗ,400,278,petrol,manual,2021-10-05,2006,1118,NaN
